### 動作確認用  

In [ ]:
# coding:utf-8
import os
import glob
import pickle
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
# 解析基準位置に移動する関数
def move_root_dir(root_dir):
    while not (os.path.basename(os.getcwd())== root_dir):
        os.chdir('../')
    print('moving to:',os.path.basename(os.getcwd()))
    return os.getcwd()

# 関数ボックス
# 読み込み
def read_csv_file(csv_file):
    df = pd.read_csv(csv_file)
    return df

# アノテーションデータをイベント変換
def get_event(df,colno,colname):
    # colno  　数値ラベル
    # colname　ラベル名
    label_box=[]
    sst=0
    sfn=0
    label=0
    nno=0
    for i in range(len(df)-1):
        a1=df.at[i  ,colno]
        a2=df.at[i+1,colno]
        if(i==0):
            sst=i
            label=a1
        if(a1!=a2 or i==len(df)-1):
            label=a1
            sfn=i
            nno=nno+1
            labelname=df.at[i,colname]
            b=[nno, sst,sfn, label ,labelname]
            label_box.append(b)
            sst=i+1
    df_event=pd.DataFrame(label_box,columns=['no','st','fn','label','state'])
    return df_event

def side_block_9(xmax,xmin,ymax,ymin,zmax,zmin):
    box_9=[]

    xmax_=max([xmax,xmin])
    xmin_=min([xmax,xmin])
    xmax = xmax_
    xmin = xmin_
    ymax_=max([ymax,ymin])
    ymin_=min([ymax,ymin])
    ymax = ymax_
    ymin = ymin_
    zmax_=max([zmax,zmin])
    zmin_=min([zmax,zmin])
    zmax = zmax_
    zmin = zmin_    
    
    x03= xmin
    y03= ymin
    z03= zmin
    x13= xmin + int(abs(xmax-xmin)/3)
    y13= ymin + int(abs(ymax-ymin)/3)
    z13= zmin + int(abs(zmax-zmin)/3)
    x23= xmin + int(abs(xmax-xmin)*2/3)
    y23= ymin + int(abs(ymax-ymin)*2/3)
    z23= zmin + int(abs(zmax-zmin)*2/3)
    x33= xmax
    y33= ymax
    z33= zmax

    # 1
    cx= int((x23 + x33)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x23,y23,z23, x23,y33,z33, x33,y23,z23, x33,y33,z33, ])    
    # 2
    cx= int((x13 + x23)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x13,y23,z23, x13,y33,z33, x23,y23,z23, x23,y33,z33, ])
    # 3
    cx= int((x03 + x13)/2.)
    cy= int((y23 + y33)/2.)
    cz= int((z23 + z33)/2.)
    box_9.append([cy,cy,cz, x03,y23,z23, x03,y33,z33, x13,y23,z23, x13,y33,z33, ])    

    # 4
    cx= int((x23 + x33)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)
    box_9.append([cy,cy,cz, x23,y13,z13, x23,y23,z23, x33,y13,z13, x33,y23,z23, ])
    # 5
    cx= int((x13 + x23)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)
    box_9.append([cy,cy,cz, x13,y13,z13, x13,y23,z23, x23,y13,z13, x23,y23,z23, ])
    # 6
    cx= int((x03 + x13)/2.)
    cy= int((y13 + y23)/2.)
    cz= int((z13 + z23)/2.)    
    box_9.append([cy,cy,cz, x03,y13,z13, x03,y23,z23, x13,y13,z13, x13,y23,z23, ])    

    # 7
    cx= int((x23 + x33)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x23,y03,z03, x23,y13,z13, x33,y03,z03, x33,y13,z13, ])    
    # 8
    cx= int((x13 + x23)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x13,y03,z03, x13,y13,z13, x23,y03,z03, x23,y13,z13, ])
    # 9
    cx= int((x03 + x13)/2.)
    cy= int((y03 + y13)/2.)
    cz= int((z03 + z13)/2.)
    box_9.append([cy,cy,cz, x03,y03,z03, x03,y13,z13, x13,y03,z03, x13,y13,z13, ])

    return box_9



In [ ]:
analysis_code='20201203_Movig_Stable_in_20201015'
root_dir = '2020_moox_morikoro_project'
root_path = move_root_dir(root_dir)

input_pkl_dir = 'preprocess_data/20201015_preprocess_data/pkl'
save_pkl_dir= 'preprocess_data/20201015_preprocess_data/pkl'
if not os.path.isdir(save_pkl_dir):
    os.makedirs(save_pkl_dir)
save_csv_dir= 'preprocess_data/20201015_preprocess_data/temp/'+analysis_code
if not os.path.isdir(save_csv_dir):
    os.makedirs(save_csv_dir)

save_fig_dir = 'output/20201015_output_data/fig/'+analysis_code
if not os.path.isdir(save_fig_dir):
    os.makedirs(save_fig_dir)

## ライブラリ読み込み   

In [ ]:
from src.analysis.labbox.calc_offline.Estimate_Stable_flag import Estimate_Stable_flag 
ESF = Estimate_Stable_flag()

# データ読み込み  

In [ ]:
# データ読み込み
# 使用軸
#col='data_fast_status_look_points_look_point_center_x'
#col='data_fast_status_look_points_look_point_center_y'
#col='data_status_face_direction_face_direction_horizontal'
#col='data_status_face_direction_face_direction_vertical'
#col='Gaze point X'
#col='Gaze point Y'

data_code = "20201015_preprocess_data"
finename = input_pkl_dir +'/'+ data_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy
print(finename)

## データ計算  

In [ ]:
data_code = "20201015_dev01_data"
re_box=[]
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 計算
    sum_window=4
    col_window=10
    threshold=0.1
    is_test = False
    ESF = Estimate_Stable_flag(sum_window=sum_window, col_window=col_window, threshold=threshold)
    col_in='data_status_face_direction_face_direction_horizontal'
    col_out="face_dir_interest_flag_horizontal"
    df=ESF.run(df, col_in, col_out, is_test)
    
    # csv保存
    filecode=save_csv_dir+'/M_'+pp+ss+tt+'_'+str(is_mask)+".csv"
    df.to_csv(filecode,index=False)
    print('save_file ',filecode)
    
    # 確保
    b=[pp,ss,tt,st,fn,df,is_mask]
    re_box.append(b)

data_box=re_box

finename = save_pkl_dir+'/'+data_code+'.pkl'
with open(finename, 'wb') as web:
    pickle.dump(data_box, web)
print('saved ', finename)

## データ解析  

In [ ]:
# 視線の静動判定  　
exp_code = "Estimate_Stable_in_20201015_exp"

xmin=-100
xmax= 3000

ic=0
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()   
    
    # アノテーションデータ可視化用処理
    ## user_result_action_label_no
    def get_event(df,colno,colname):    
        label_box=[]
        sst=0
        sfn=0
        label=0
        nno=0
        for i in range(len(df)-1):
            a1=df.at[i  ,colno]
            a2=df.at[i+1,colno]
            if(i==0):
                sst=i
                label=a1
            if(a1!=a2 or i==len(df)-1):
                label=a1
                sfn=i
                nno=nno+1
                labelname=df.at[i,colname]
                b=[nno, sst,sfn, label ,labelname]
                label_box.append(b)
                sst=i+1
        df_event=pd.DataFrame(label_box,columns=['no','st','fn','label','state'])
        return df_event
    colno='user_result_action_label_no'
    colname='user_result_action_label_name'
    df_event=get_event(df,colno,colname)
    colno='interest_label_no'
    colname='interest_label_name'
    df_event2=get_event(df,colno,colname)
    colno='face_dir_interest_flag_horizontal'
    colname='face_dir_interest_flag_horizontal_name'
    df.loc[(df[colno] ==0),colname] = 'estimate_interest'
    df.loc[(df[colno] ==1),colname] = 'estimate_not_interest'
    df_event4=get_event(df,colno,colname)
    
    

    # 結果可視化
    print()
    print('-----------------------------------------------------------------------')
    print()
    mess="静動判定フラグ"
    print(mess)
    title=exp_code+'_estimate_interest_flag_stable1_moving0'
    def plot_fig(df,title,save_fig_dir,xmin,xmax):    
        # 描画　停留状態の強調
        plt.figure(figsize=(20,4))
        col='face_dir_interest_flag_horizontal'
        df[col]=df[col].fillna(0)
        x=df[col]
        plt.plot(x,label=col,color='b',alpha=0.2)
        # 描画
        plt.title(title)
        plt.ylim(-0.5,1.5)
        plt.xlim(xmin,xmax)
        plt.ylabel("flag")
        plt.xlabel("Time 0.1 [s]")
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_fig_dir, save_file)
        plt.savefig(save_path)
        plt.show() 
    plot_fig(df,title,save_fig_dir,xmin,xmax)
    print()
    print('-----------------------------------------------------------------------')
    print()
    # 行動との対応関係
    mess="静動フラグから静止状態を抽出した点のプロットとattention状態の比較\n青色帯はattention状態を示す"
    print(mess)
    title=exp_code+'_stable_look_h_and_attention'
    def plot_fig(df,title,save_fig_dir,xmin,xmax,df_event):
        plt.figure(figsize=(20,4))
        col='data_status_face_direction_face_direction_horizontal'
        #col_out="face_dir_ms_flag_horizontal"
        col_out="face_dir_interest_flag_horizontal"
        x=df[col]
        plt.plot(x,label=col,color='r',alpha=0.2)
        df.loc[(df[col_out] == 0),col_out] = np.nan
        x=df[col] * df[col_out]
        plt.plot(x,label='Stable_Point',color='r',marker=".",linestyle='None')
        # 行動ラベル
        df_event=df_event.loc[df_event['label']==4.0]
        df_event=df_event.reset_index(drop=True)
        for i in range(len(df_event)):
            sti=df_event.at[i,'st']
            fni=df_event.at[i,'fn']
            label=df_event.at[i,'label']
            state=df_event.at[i,'state']
            #print(state)
            plt.axvspan(sti, fni, color ='b', alpha=0.1)
        col='user_result_action_label_no'
        x=df[col]*(-10)-30
        plt.plot(x,label=col,color='b',alpha=0.4)
        # 描画
        plt.title(title)
        plt.ylim(-80,80)
        plt.xlim(xmin,xmax)
        plt.ylabel("face_direction_horizontal [degree]")
        plt.xlabel("Time 0.1 [s]")
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_fig_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_fig(df,title,save_fig_dir,xmin,xmax,df_event)
    print()
    print('-----------------------------------------------------------------------')
    print()   
    mess="静動フラグから静止状態を抽出した点のプロットとfollow状態の比較\n緑色帯はfollow状態を示す"
    print(mess)
    title=exp_code+'_stable_look_h_and_follow'
    def plot_fig(df,title,save_fig_dir,xmin,xmax,df_event):
        plt.figure(figsize=(20,4))
        col='data_status_face_direction_face_direction_horizontal'
        #col_out="face_dir_ms_flag_horizontal"
        col_out="face_dir_interest_flag_horizontal"
        x=df[col]
        plt.plot(x,label=col,color='r',alpha=0.2)
        df.loc[(df[col_out] == 0),col_out] = np.nan
        x=df[col] * df[col_out]
        plt.plot(x,label='Stable_Point',color='r',marker=".",linestyle='None')
        # 行動ラベル
        df_event=df_event.loc[df_event['label']==3.0]
        df_event=df_event.reset_index(drop=True)
        for i in range(len(df_event)):
            sti=df_event.at[i,'st']
            fni=df_event.at[i,'fn']
            label=df_event.at[i,'label']
            state=df_event.at[i,'state']
            #print(state)
            plt.axvspan(sti, fni, color ='g', alpha=0.1)
        col='user_result_action_label_no'
        x=df[col]*(-10)-30
        plt.plot(x,label=col,color='b',alpha=0.4)
        # 描画
        plt.title(title)
        plt.ylim(-80,80)
        plt.xlim(xmin,xmax)
        plt.ylabel("face_direction_horizontal [degree]")
        plt.xlabel("Time 0.1 [s]")
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_fig_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_fig(df,title,save_fig_dir,xmin,xmax,df_event)
    print()
    print('-----------------------------------------------------------------------')
    print()
    mess="静動フラグから静止状態を抽出した点のプロットとlooking_around状態の比較\n赤色帯はlooking_around状態を示す"
    print(mess)    
    title=exp_code+'_stable_look_h_and_looking_around'
    def plot_fig(df,title,save_fig_dir,xmin,xmax,df_event):
        plt.figure(figsize=(20,4))
        col='data_status_face_direction_face_direction_horizontal'
        #col_out="face_dir_ms_flag_horizontal"
        col_out="face_dir_interest_flag_horizontal"
        x=df[col]
        plt.plot(x,label=col,color='r',alpha=0.2)
        df.loc[(df[col_out] == 0),col_out] = np.nan
        x=df[col] * df[col_out]
        plt.plot(x,label='Stable_Point',color='r',marker=".",linestyle='None')
        # 行動ラベル
        df_event=df_event.loc[df_event['label']==2.0]
        df_event=df_event.reset_index(drop=True)
        for i in range(len(df_event)):
            sti=df_event.at[i,'st']
            fni=df_event.at[i,'fn']
            label=df_event.at[i,'label']
            state=df_event.at[i,'state']
            #print(state)
            plt.axvspan(sti, fni, color ='R', alpha=0.1)
        col='user_result_action_label_no'
        x=df[col]*(-10)-30
        plt.plot(x,label=col,color='b',alpha=0.4)
        # 描画
        plt.title(title)
        plt.ylim(-80,80)
        plt.xlim(xmin,xmax)
        plt.ylabel("face_direction_horizontal [degree]")
        plt.xlabel("Time 0.1 [s]")
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_fig_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_fig(df,title,save_fig_dir,xmin,xmax,df_event)    
    print()
    print('-----------------------------------------------------------------------')
    print()
    mess="静動フラグから静止状態を抽出した点のプロットと推定した注視状態の比較\n黄色帯は推定注視状態を示す"
    print(mess)
    title=exp_code+'_stable_look_h_and_estimate_interest'
    def plot_fig(df,title,save_fig_dir,xmin,xmax,df_event):
        plt.figure(figsize=(20,4))
        col='data_status_face_direction_face_direction_horizontal'
        col_out='face_dir_interest_flag_horizontal'
        #col_out="face_dir_ms_flag_horizontal"
        x=df[col]
        plt.plot(x,label=col,color='r',alpha=0.2)
        df.loc[(df[col_out] == 0),col_out] = np.nan
        x=df[col] * df[col_out]
        plt.plot(x,label='Stable_Point',color='r',marker=".",linestyle='None')
        # 行動ラベル
        df_event=df_event.loc[df_event['label']==1.0]
        df_event=df_event.reset_index(drop=True)
        for i in range(len(df_event)):
            sti=df_event.at[i,'st']
            fni=df_event.at[i,'fn']
            label=df_event.at[i,'label']
            state=df_event.at[i,'state']
            #print(state)
            plt.axvspan(sti, fni, color ='y', alpha=0.2)
        col='interest_label_no'
        x=df[col]*(-10)-50
        plt.plot(x,label=col,color='g',alpha=0.4)
        # 描画
        plt.title(title)
        plt.ylim(-80,80)
        plt.xlim(xmin,xmax)
        plt.ylabel("face_direction_horizontal [degree]")
        plt.xlabel("Time 0.1 [s]")
        plt.legend(loc="upper left",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_fig_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_fig(df,title,save_fig_dir,xmin,xmax,df_event4)
    print()
    print('-----------------------------------------------------------------------')
    print()
    # confmat
    print('interest_label と静動判定の精度評価')
    y_true=df['interest_label_no']
    y_pred=df['face_dir_interest_flag_horizontal'].fillna(0)
    cm=confusion_matrix(y_true, y_pred)
    s=np.sum(cm)
    print('confmat rate')
    print(cm/s)
    print('confmat')
    print(confusion_matrix(y_true, y_pred))
    print('precision',precision_score(y_true, y_pred))
    print('recall',recall_score(y_true, y_pred))
    print('f1',f1_score(y_true, y_pred))  
    

In [ ]:
# Tobiiとの比較　　
exp_code = "Estimate_Stable_in_20201015_exp"

#xmin=-100
#xmax=3000
xmin=-100
xmax= 3000
ic=0
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy() 
    
    # tobii 情報の編集
    col_in='Eye movement type'
    col_out='Eye movement type No'
    state_dict={
        'EyesNotFound':np.nan,
        'Fixation':2,
        'Saccade':3,
        'Unclassified':np.nan,
        }
    for i in range(len(df)):
        state=df.at[i,col_in]
        df.at[i,col_out]=state_dict[state]
    
    # アノテーションデータ可視化用処理
    ## user_result_action_label_no
    colno='user_result_action_label_no'
    colname='user_result_action_label_name'
    df_event=get_event(df,colno,colname)
    #print(df_event)
    colno='interest_label_no'
    colname='interest_label_name'
    df_event2=get_event(df,colno,colname)
    #print(df_event2)
    colno='face_dir_interest_flag_horizontal'
    colname='face_dir_interest_flag_horizontal_name'
    df.loc[(df[colno] ==0),colname] = 'estimate_interest'
    df.loc[(df[colno] ==1),colname] = 'estimate_not_interest'
    df_event4=get_event(df,colno,colname)
    # tobii イベント化
    col_in='Eye movement type'
    colno='Eye movement type No'
    colname='Eye movement type'
    df.loc[(df[colno] ==0),colname] = 'Unclassified'
    df.loc[(df[colno] ==1),colname] = 'EyesNotFound'
    df.loc[(df[colno] ==2),colname] = 'Fixation'
    df.loc[(df[colno] ==3),colname] = 'Saccade'
    df_event5=get_event(df,colno,colname)
    # print(df_event5)
    print()
    print('-----------------------------------------------------------------------')
    print()
    # Tobii と評価
    print()
    print('-----------------------------------------------------------------------')
    print()
    mess="推定した注視状態の比較(黄色)とtobiiによる注視状態(Fixation)の比較"
    print(mess)
    title=exp_code+'_Tobii_gaze_X_and_tobii_Eye_movement_type'
    def plot_fig(df,title,save_fig_dir,xmin,xmax,df_event,df_event2):
        plt.figure(figsize=(20,8))
        col='Gaze point X'
        x=df[col]
        plt.plot(x,label=col,color='r',alpha=0.6)
        # tobii 判定ラベル
        col='Eye movement type No'
        x=df[col]*(-300)+700
        plt.plot(x,label=col,color='g',alpha=1.0)
        plt.text(0,  120, 'Tobii Fixation',fontsize=16,color='g')
        plt.text(0, -300, 'Tobii Saccade',fontsize=16,color='g')
        #
        col='interest_label_no'
        x=df[col]*(300)-800
        plt.plot(x,label=col,color='y',alpha=1.0)
        plt.text(0, -450, 'Recognition on Interest',fontsize=16,color='y')
        plt.text(0,-900, 'Recognition on not Interest',fontsize=16,color='y')        
        # 描画
        plt.title(title, fontsize=18)
        plt.ylim(-1000,1750)
        plt.yticks([1500,1000,500,], ["1500","1000","500",])
        plt.xlim(xmin,xmax)
        plt.ylabel("Feature index   and  Tbii gaze x [pixel]", fontsize=16)
        plt.xlabel("Time 0.1 [s]", fontsize=16)
        plt.legend(loc="upper right",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_fig_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_fig(df,title,save_fig_dir,xmin,xmax,df_event4,df_event5)
    print()
    print('-----------------------------------------------------------------------')
    print()
    # confmat
    col1='interest_label_no'
    col2_in='Eye movement type No'
    col2='Tobii Eye movement type no'
    df.loc[(df[col2_in] ==0),col2] = np.nan
    df.loc[(df[col2_in] ==1),col2] = np.nan
    df.loc[(df[col2_in] ==2),col2] = 1
    df.loc[(df[col2_in] ==3),col2] = 0
    
    y_true=df[col1]
    y_pred=df[col2].fillna(0)
    cm=confusion_matrix(y_true, y_pred)
    s=np.sum(cm)
    print('confmat rate',cm/s)
    print('confmat',confusion_matrix(y_true, y_pred))
    print('precision',precision_score(y_true, y_pred))
    print('recall',recall_score(y_true, y_pred))
    print('f1',f1_score(y_true, y_pred)) 
    
    
    # tobii と　提案手法の比較
    print()
    print('-----------------------------------------------------------------------')
    print()
    mess="推定した注視状態の比較(黄色)とtobiiによる注視状態(Fixation)の比較"
    print(mess)
    title=exp_code+'_stable_look_dir_horizontal_and_tobii_Eye_movement_type'
    def plot_fig(df,title,save_fig_dir,xmin,xmax,df_event,df_event2):
        plt.figure(figsize=(20,8))
        col='data_status_face_direction_face_direction_horizontal'
        col_out='face_dir_interest_flag_horizontal'
        #col_out="face_dir_ms_flag_horizontal"
        x=df[col]
        plt.plot(x,label=col,color='r',alpha=0.4)
        df.loc[(df[col_out] == 0),col_out] = np.nan
        x=df[col] * df[col_out]
        plt.plot(x,label='Stable_Point',color='r',marker=".",linestyle='None')
        # 行動ラベル　detect stable
        col='face_dir_interest_flag_horizontal'
        df[col]=df[col].fillna(0)
        x=df[col]*(40)-90
        plt.plot(x,label=col,color='b',alpha=1.0)
        plt.text(0, -45, 'estimate Stable',fontsize=16,color='b')
        plt.text(0,-100, 'estimate Moving',fontsize=16,color='b')
        # tobii 判定ラベル
        col='Eye movement type No'
        x=df[col]*(-40)-40
        plt.plot(x,label=col,color='g',alpha=1.0)
        plt.text(0, -170, 'Tobii Saccade',fontsize=16,color='g')
        plt.text(0,-115, 'Tobii Fixation',fontsize=16,color='g')
        #
        col='interest_label_no'
        x=df[col]*(40)-230
        plt.plot(x,label=col,color='y',alpha=1.0)
        plt.text(0, -185, 'Recognition on Interest',fontsize=16,color='y')
        plt.text(0,-240, 'Recognition on not Interest',fontsize=16,color='y')        
        # 描画
        plt.title(title, fontsize=18)
        plt.ylim(-250,80)
        plt.yticks([40,20,0,-20,-40], ["40","20","0","- 20","- 40"])
        plt.xlim(xmin,xmax)
        plt.ylabel("Feature index   and  face_direction_horizontal [degree]", fontsize=16)
        plt.xlabel("Time 0.1 [s]", fontsize=16)
        plt.legend(loc="upper right",frameon=False)
        # save as png
        save_file=title+'.png'
        save_path=os.path.join(save_fig_dir, save_file)
        plt.savefig(save_path)
        plt.show()
    plot_fig(df,title,save_fig_dir,xmin,xmax,df_event4,df_event5)
    print()
    print('-----------------------------------------------------------------------')
    print() 
    # confmat
    col1='face_dir_interest_flag_horizontal'
    col2_in='Eye movement type No'
    col2='Tobii Eye movement type no'
    df.loc[(df[col2_in] ==0),col2] = np.nan
    df.loc[(df[col2_in] ==1),col2] = np.nan
    df.loc[(df[col2_in] ==2),col2] = 1
    df.loc[(df[col2_in] ==3),col2] = 0
    
    y_true=df[col1]
    y_pred=df[col2].fillna(0)
    cm=confusion_matrix(y_true, y_pred)
    s=np.sum(cm)
    print('confmat rate',cm/s)
    print('confmat',confusion_matrix(y_true, y_pred))
    print('precision',precision_score(y_true, y_pred))
    print('recall',recall_score(y_true, y_pred))
    print('f1',f1_score(y_true, y_pred)) 